## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [52]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.describe()


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [53]:
print(df.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

Описание данных:

children — количество детей в семье

days_employed — общий трудовой стаж в днях

dob_years — возраст клиента в годах

education — уровень образования клиента

education_id — идентификатор уровня образования

family_status — семейное положение

family_status_id — идентификатор семейного положения

gender — пол клиента

income_type — тип занятости

debt — имел ли задолженность по возврату кредитов

total_income — ежемесячный доход

purpose — цель получения кредита

In [54]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод


При первом просмотре данных была выявлены ошибки:
1. Отрицательный рабочий стаж
2. Пропущенные значения
3. Дубликаты
4. Слова в разном регистре

Для того, чтобы узнать, какие ошибки еще есть, воспользуемся методами выявления и подсчета таких ошибок, а потом устаним их.

# Шаг 2. Предобработка данных

На втором шаге мы выполним следующие действия:
1. определим и заполним пропущенные значения:
2. опишем, какие пропущенные значения обнаружили;
3. приведем возможные причины появления пропусков в данных;
4. объясним, по какому принципу заполнены пропуски

### Обработка пропусков

С помощью метода .isna() определим, в каких столбцах встречаются пропущенные значения.

In [55]:
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Пропущенные значения встречаются в столбцах days_employed (трудовой стаж) и total_income (ежемесячный доход). Можно предположить, что пропуски и в том, и в другом столбце связаны с конвертацией данных из одного формата в другой или из-за потобной технической нестыковки.
Далее посмотрим, вызвав к столбцу 'days_employed', 

Чтобы заполнить пропуски в столбце days_employed, заменим пропущенные значения медианой.

In [56]:
df['days_employed'] = df['days_employed'].abs()

In [57]:
days_emp_med = df['days_employed'].median()
print(days_emp_med)

2194.220566878695


In [58]:
print(df[df['days_employed'].isna() == True])

       children  days_employed  dob_years education  education_id  \
12            0            NaN         65   среднее             1   
26            0            NaN         41   среднее             1   
29            0            NaN         63   среднее             1   
41            0            NaN         50   среднее             1   
55            0            NaN         54   среднее             1   
...         ...            ...        ...       ...           ...   
21489         2            NaN         47   Среднее             1   
21495         1            NaN         50   среднее             1   
21497         0            NaN         48    ВЫСШЕЕ             0   
21502         1            NaN         42   среднее             1   
21510         2            NaN         28   среднее             1   

               family_status  family_status_id gender  income_type  debt  \
12          гражданский брак                 1      M    пенсионер     0   
26           женат 

In [59]:
df['days_employed'] = df['days_employed'].fillna(days_emp_med)
print(df.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42               высшее             0   
1          1    4024.803754         36              среднее             1   
2          0    5623.422610         33              Среднее             1   
3          3    4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0     926.185831         27               высшее             0   
6          0    2879.202052         43               высшее             0   
7          0     152.779569         50              СРЕДНЕЕ             1   
8          2    6929.865299         35               ВЫСШЕЕ             0   
9          0    2188.756445         41              среднее             1   
10         2    4171.483647         36               высшее             0   
11         0     792.701887         40              среднее             1   

In [60]:
df.loc[df['days_employed'] == 'NaN', 'days_employed'] = days_emp_med
print(df.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42               высшее             0   
1          1    4024.803754         36              среднее             1   
2          0    5623.422610         33              Среднее             1   
3          3    4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0     926.185831         27               высшее             0   
6          0    2879.202052         43               высшее             0   
7          0     152.779569         50              СРЕДНЕЕ             1   
8          2    6929.865299         35               ВЫСШЕЕ             0   
9          0    2188.756445         41              среднее             1   
10         2    4171.483647         36               высшее             0   
11         0     792.701887         40              среднее             1   

Таже в данной таблице мы видим отрицательный рабочий стаж, который следует поставить по модулю методом .abs()

In [61]:
df['days_employed'] = df['days_employed'].abs()

In [62]:
print(df['days_employed'])

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64


Теперь заменим все пропуски в столбце с ежемесячным доходом на медианное значение. Так как мы располагаем данными клиентов, которые уже получили кредит, данные клиентов ценны и их пропуск не является случайным и влияет на наш дальнейший анализ.

In [63]:
m_table = df.groupby(['income_type']).agg({'total_income' : 'median'})
for income_type in m_table.index:
    w=m_table.loc[income_type, 'total_income']
    df.loc[df['income_type'] == income_type, 'total_income'] = df.loc[df['income_type'] == income_type, 'total_income'].fillna(w)
print(df[['income_type', 'total_income']].head(20))

   income_type   total_income
0    сотрудник  253875.639453
1    сотрудник  112080.014102
2    сотрудник  145885.952297
3    сотрудник  267628.550329
4    пенсионер  158616.077870
5    компаньон  255763.565419
6    компаньон  240525.971920
7    сотрудник  135823.934197
8    сотрудник   95856.832424
9    сотрудник  144425.938277
10   компаньон  113943.491460
11   сотрудник   77069.234271
12   пенсионер  118514.486412
13   сотрудник  130458.228857
14   компаньон  165127.911772
15   сотрудник  116820.904450
16   сотрудник  289202.704229
17   сотрудник   90410.586745
18   пенсионер   56823.777243
19   сотрудник  242831.107982


Теперь проверим, все ли пропуски мы заполнили.

In [64]:
print(df.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Теперь все в порядке, можно анализировать данные дальше.

### Вывод

Методом подсчета пропусков было выявлено, что клиенты пропускают/не заполняют графы со своим доходом и рабочим стажем. Но с помощью методов .isna() и .fillna() мы устанили данную проблему.

### Замена типа данных

На данном этапе посмотрим, есть ли в столбцах нечисловые значения или пропуски, которые могут помешать нашим расчётам.

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Для удобства переведем значения ежемесячного дохода в целочисленные значения методом .round().

In [66]:
df['total_income'] = df['total_income'].round()
print(df['total_income'])

0        253876.0
1        112080.0
2        145886.0
3        267629.0
4        158616.0
           ...   
21520    224792.0
21521    156000.0
21522     89673.0
21523    244093.0
21524     82047.0
Name: total_income, Length: 21525, dtype: float64


Теперь избавимся от вещественного типа чисел в столбцах.

In [67]:
df['total_income'] = df['total_income'].astype('int')
print(df['total_income'])

0        253876
1        112080
2        145886
3        267629
4        158616
          ...  
21520    224792
21521    156000
21522     89673
21523    244093
21524     82047
Name: total_income, Length: 21525, dtype: int64


In [68]:
df['days_employed'] = df['days_employed'].astype('int')
print(df['days_employed']) 

0          8437
1          4024
2          5623
3          4124
4        340266
          ...  
21520      4529
21521    343937
21522      2113
21523      3112
21524      1984
Name: days_employed, Length: 21525, dtype: int64


### Вывод

Был устранен вещественный тип данных в столбце  total_income, а также выявлен аналогичный тип данных в days_employed, но сейчас его устранить не удалось. Ясно, что вещественныый тип данных в столбцах с числами мог появиться из-за наличия там NaNов, которых мы успешно устранили на предыдущем этапе.

Как можно избавиться от слова 'median'в столбце days_employed, и стоит ли это делать. Возможно, стоит оставить это значение как артефакт.

### Обработка дубликатов

In [69]:
print(df.duplicated().sum())

54


С помощью метода подсчета дубликатов было выявлено 54 дубликата, которые мы будем устранять и разбирать на следующем этапе.

In [70]:
df = df.drop_duplicates().reset_index(drop=True)

In [71]:
print(df.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1           8437         42               высшее             0   
1          1           4024         36              среднее             1   
2          0           5623         33              Среднее             1   
3          3           4124         32              среднее             1   
4          0         340266         53              среднее             1   
5          0            926         27               высшее             0   
6          0           2879         43               высшее             0   
7          0            152         50              СРЕДНЕЕ             1   
8          2           6929         35               ВЫСШЕЕ             0   
9          0           2188         41              среднее             1   
10         2           4171         36               высшее             0   
11         0            792         40              среднее             1   

В столбце education мы видим слова в разном регистре, это также является причиной дубликатов, устраним.

In [72]:
df['education'] = df['education'].str.lower()

In [73]:
print(df['education'].head(20))

0                  высшее
1                 среднее
2                 среднее
3                 среднее
4                 среднее
5                  высшее
6                  высшее
7                 среднее
8                  высшее
9                 среднее
10                 высшее
11                среднее
12                среднее
13    неоконченное высшее
14                 высшее
15                среднее
16                среднее
17                 высшее
18                среднее
19                среднее
Name: education, dtype: object


Приверим, все ли пропуски мы устранили.

In [74]:
print(df.duplicated().sum())

17


In [75]:
df = df.drop_duplicates().reset_index(drop=True)
print(df.head(20))

    children  days_employed  dob_years            education  education_id  \
0          1           8437         42               высшее             0   
1          1           4024         36              среднее             1   
2          0           5623         33              среднее             1   
3          3           4124         32              среднее             1   
4          0         340266         53              среднее             1   
5          0            926         27               высшее             0   
6          0           2879         43               высшее             0   
7          0            152         50              среднее             1   
8          2           6929         35               высшее             0   
9          0           2188         41              среднее             1   
10         2           4171         36               высшее             0   
11         0            792         40              среднее             1   

In [76]:
print(df.duplicated().sum())

0


### Вывод

На этом шаге были удалены все дубликаты с помощью .drop_duplicates() и с помошью reset_index(drop=True) был восстановлен правильный порядок значений в таблице, т.е. теперь таблица выглядит также, как и до изменений, но в ней отсутствуют дубликаты.
Также был выполнен ручной поиск скрытых дубликатов в столбце с образованием - все слова теперь в нижнем регистре, что не мешает анализировать строки.

### Лемматизация

выделите леммы в значениях столбца с целями получения кредита:
опишите, как вы проводили лемматизацию целей кредита;

Для работы с леммами воспользуемся библиотекой PyMystem.

In [77]:
dictionary = pd.DataFrame(data = df['purpose'], columns=['purpose'])
print(dictionary)

                            purpose
0                     покупка жилья
1           приобретение автомобиля
2                     покупка жилья
3        дополнительное образование
4                   сыграть свадьбу
...                             ...
21449             операции с жильем
21450          сделка с автомобилем
21451                  недвижимость
21452  на покупку своего автомобиля
21453         на покупку автомобиля

[21454 rows x 1 columns]


In [78]:
from pymystem3 import Mystem
m = Mystem()

def purpose_category(row):
    
    lemmas = m.lemmatize(row)
    
    if ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'недвижимость'
    
    elif 'образование' in lemmas:
        return 'образование'
    
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    
    elif 'свадьба' in lemmas:
        return 'свадьба'
    
dictionary['category'] = dictionary['purpose'].apply(purpose_category)

print(dictionary['category'])
df['category'] = df['purpose'].apply(purpose_category)
print(df)


0        недвижимость
1          автомобиль
2        недвижимость
3         образование
4             свадьба
             ...     
21449    недвижимость
21450      автомобиль
21451    недвижимость
21452      автомобиль
21453      автомобиль
Name: category, Length: 21454, dtype: object
       children  days_employed  dob_years education  education_id  \
0             1           8437         42    высшее             0   
1             1           4024         36   среднее             1   
2             0           5623         33   среднее             1   
3             3           4124         32   среднее             1   
4             0         340266         53   среднее             1   
...         ...            ...        ...       ...           ...   
21449         1           4529         43   среднее             1   
21450         0         343937         67   среднее             1   
21451         1           2113         38   среднее             1   
21452         3        

### Вывод

С помощью леммизиции мы выявили, что существует не так уж и много целей для того, чтобы брать кредит. Теперь цели выглядят однообразно, что позволит нам их сравнивать.

### Категоризация данных

Категоризацию необходимо применить, чтобы увидеть, какие слои населения берут кредиты и сравнивать их по группам. Для этого разделим клиентов на низший (poor), средний(average) и зажиточный (rich) классы и составим цикл, который будет распределять клиентов по этим категориям.

In [79]:
poor = df['total_income'].quantile(0.25)
print(poor)

107624.0


In [80]:
average = df['total_income'].quantile(0.5)
print(average)

142594.0


In [81]:
rich = df['total_income'].quantile(0.75)
print(rich)

195821.25


In [82]:
income_dict = pd.DataFrame(data = df.total_income.unique(), columns = ['total_income'])

def income_capacity(row):
    
    income = row['total_income']
    
    if income <= poor:
        return 'маленький доход'
    
    if (income > poor) and (income <= average):
        return 'средний доход'
    
    if income > average:
        return 'высокий доход'
    
    
row_values = df['total_income'].sum()
row_columns = ['total_income']
row = pd.Series(data=row_values, index=row_columns)

print(income_capacity(row))
    

высокий доход


Добавим новый столбец с категорией дохода в нашу таблицу:

In [83]:
df['income_capacity'] = df.apply(income_capacity, axis=1)
print(df)

       children  days_employed  dob_years education  education_id  \
0             1           8437         42    высшее             0   
1             1           4024         36   среднее             1   
2             0           5623         33   среднее             1   
3             3           4124         32   среднее             1   
4             0         340266         53   среднее             1   
...         ...            ...        ...       ...           ...   
21449         1           4529         43   среднее             1   
21450         0         343937         67   среднее             1   
21451         1           2113         38   среднее             1   
21452         3           3112         38   среднее             1   
21453         2           1984         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

### Вывод

Таким образом было выявлено, что большая часть клиентов принадлежит к среднему классу. Это говорит о то, что средний класс нуждается в средствах больше, чем зажиточный класс, и может позволить себе взять кредит и скорее его выплатит, чем бедный класс населения.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для того, чтобы ответить на этот вопрос, построим таблицу по наличии детей и наличия задолжности.

In [84]:
table1 = df.pivot_table(index='children' ,columns='debt', aggfunc='count')['days_employed']
print(table1)

debt            0       1
children                 
-1           46.0     1.0
 0        13028.0  1063.0
 1         4364.0   444.0
 2         1858.0   194.0
 3          303.0    27.0
 4           37.0     4.0
 5            9.0     NaN
 20          68.0     8.0


В этой таблицы мы видим некоторые ошибки:

- 20 - в количестве детей - это явно грубая ошибка - оставим 2 вместо 20.

- NaN в количестве человек, у которых 5 детей и неоплаченный вовремя долг (я считаю, что пропуск в данном случае лучше не исправлять на срединное/медианное значение, потому что мы видим большой разброс как в количестве людей с долгом, так и я думаю, что не стоит приравнивать людей с 5ю детьми и людей с 1-4 детьми. Поэтому оптимальным варинтом будет поставить вместо пропуска 0).

- отрицательное количество детей также можно списать на грубую ошибку - поставим это значение по модулю.

Посчитаем, сколько процентов бездетных клиентов возвращают кредит в срок.

In [85]:
per_without_children = 952.0 / (11758.0 + 952.0)
print(per_without_children)

0.07490165224232888


Посчитаем, сколько процентов клиентов с одним ребенком возвращают кредит в срок.

In [86]:
with_1_child = (408.0 + 1.0) / (3935.0 + 408.0 + 43.0 + 1.0)
print(with_1_child)

0.09322999772053796


In [87]:
with_2_ch = (177 + 8) / (1674.0 + 177.0 + 59.0 + 8.0)
print(with_2_ch)

0.09645464025026068


In [88]:
with_3_ch = 22 / (272.0 + 22.0)
print(with_3_ch)

0.07482993197278912


In [89]:
with_4_ch = 3 / (31.0 + 3.0)
print(with_4_ch)

0.08823529411764706


In [90]:
with_5_ch = 0 / (8 + 0)
print(with_5_ch)  #данный расчет требует дополнительных данных

0.0


### Вывод

Проанализировав соотношение долгов у клиентов с детьми и бездетных клиентов, мы выявили интересные результаты. Наиболее стабильными плательщиками являются люди без детей и с тремя детьми. 
Что касается клиентов с количеством детей от 5, для более глубокого изучения данной категории требуеются дополнительные данные (больше выборка и т.д.).

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [92]:
table2 = df.pivot_table(index='family_status', columns='debt', aggfunc='count')['days_employed']
print(table2)

debt                       0    1
family_status                    
Не женат / не замужем   2536  274
в разводе               1110   85
вдовец / вдова           896   63
гражданский брак        3763  388
женат / замужем        11408  931


Далее посчитаем соотношение тех, у кого есть долг, к общему количеству клиентов данной категории.

In [93]:
not_married = 254 / (2271 + 254)
print(not_married)     # не женат/не замужем

0.1005940594059406


In [94]:
divorced = 76 / (1007 + 76)
print(divorced)   #в разводе

0.07017543859649122


In [95]:
widow = 56 / (809 + 56)
print(widow)   #вдовец/вдова

0.06473988439306358


In [96]:
civil_marriage = 339 / (3396 + 339)
print(civil_marriage)   #гражданский брак

0.09076305220883533


In [97]:
married = 846 / (10297 + 846)
print(married)    #женат/замужем

0.07592210356277483


### Вывод

Проведя сравнительный анализ, было выявлено, что люди несостоящие/несостоявшие/без пары являются самыми непостоянными плательщиками - 10% людей из данной категории не выплачивает кредит в срок. А наиболее постоянными плательщиками являются вдова и вдовцы. Могу предположить, что это связано с тем, что после смерти супруга/супруги, вдовцам достается наследство, с помощбю которого они могут брать и выплачивать кредиты, но данная гипотеза требует дальнейшей обработки.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Разобьем пользователей на группы с помощью квантилей. 0.25 - poor, 0.5 - averange, 0.75 - rich

In [98]:
df.quantile([.25, .5, .75])

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
0.25,0.0,1023.0,33.0,1.0,0.0,0.0,107624.00
0.50,0.0,2194.0,42.0,1.0,0.0,0.0,142594.00
0.75,1.0,4797.0,53.0,1.0,1.0,0.0,195821.25


Подтвердим наш вывод сводной таблицей:

In [99]:
table7 = df.pivot_table(index = 'income_capacity', columns='debt', aggfunc='count')['days_employed']
print(table7)

debt                0    1
income_capacity           
высокий доход    9780  831
маленький доход  4937  427
средний доход    4996  483


Для того, чтобы сравнить полученные значения, выведем проценты тех людей по классам, у которых есть долги.

In [100]:
percent_for_rich = (831 * 100) / (9780 + 831)
print(percent_for_rich)

7.831495617755159


In [101]:
percent_for_average = (483 * 100) / (4996 + 483)
print(percent_for_average)

8.815477276875342


In [102]:
percent_for_poor = (427 * 100) / (4937 + 427)
print(percent_for_poor)

7.960477255779269


### Вывод

Как видно из сводной таблицы и вывода значений в процентах, существует зависимость между уровнем дохода и возвратом кредита в срок. Больший процент должников находится в среднем классе (8.8%).

- Как разные цели кредита влияют на его возврат в срок?

In [103]:
table4 = df.pivot_table(index= 'category',columns='debt', aggfunc='count')['days_employed']
print(table4)

debt              0    1
category                
автомобиль     3903  403
недвижимость  10029  782
образование    3643  370
свадьба        2138  186


Для того, чтобы выявить процент должников в своей категории, проведем следующие математические операции:

In [104]:
car = 403 * 100 /(3903 + 403)
print(car)

9.359033906177427


In [105]:
house = 782 * 100 /(10029 + 782)
print(house)

7.233373415965221


In [106]:
education = 370 * 100 /(3643 + 370)
print(education)

9.22003488661849


In [107]:
marriage = 186 * 100 /(2138 + 186)
print(marriage)

8.003442340791738


### Вывод

Как видно из расчетов, больший процент должников имеют цели купить машину или заплатить за образование. Можно предположить, что должники с целями получения самообразования являются студентами или закончившими первое высшее, и не могут регулярно выплачивать нужную сумму. Этот вопрос требует дополнительного анализа.

### Шаг 4. Общий вывод

После того, как был проведен анализ исходных данных с их предварительной обработкой, можно сказать, что:

- Наиболее стабильными плательщиками являются люди без детей и с тремя детьми

- Наиболее постоянными плательщиками являются вдова и вдовцы.

- Больший процент должников находится в среднем классе.

- Больше всего людей берут кредиты для получения образования и покупки машины.
Выводы по поводу данных:

1. какими бы ясными / не нуждающимися в обработке не выглядили данные, их все равно нужно проверить на ошибки, пропуски, дубликаты и т.д. И даже после этого могут при постоении различного рода таблиц выглядывать пропуски.
2. Мало применять методы, нужно вникнуть в тему исследования и мыслить логически.
3. НО несмотря на на доводы логики, результаты анализа иногда могут удивлять, что произошло, когда мы анализировали отношение между количесвом детей и выплатой кредита в срок. Если расчеты верны, и клиенты с ебольшим количеством детей являются достаточно надежными плательщиками.
